# AI Agents tools is LangChain

In [3]:
# Installing some dependencies:
%pip install duckduckgo-search langchain_experimental ddgs

  Using cached Brotli-1.1.0-cp312-cp312-win_amd64.whl.metadata (5.6 kB)
Using cached Brotli-1.1.0-cp312-cp312-win_amd64.whl (357 kB)

   ------------- -------------------------- 2/6 [hyperframe]
   -------------------- ------------------- 3/6 [hpack]
   -------------------------- ------------- 4/6 [h2]
   --------------------------------- ------ 5/6 [ddgs]
   --------------------------------- ------ 5/6 [ddgs]
   --------------------------------- ------ 5/6 [ddgs]
   ---------------------------------------- 6/6 [ddgs]

Note: you may need to restart the kernel to use updated packages.


## Built-in-tool ***DuckDuckGoSearch***

In [8]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

print(search.invoke("How many cuda cores does nvidias RTX PRO 6000 has?"))

Nvidia has released many cards that support RTX , including the 20, 30, 40, and 50 series:[14]. An OpenCL benchmark result of Nvidia ' s upcoming RTX Pro 6000 Blackwell Workstation Edition graphics card has emerged, confirming rumored specifications of the GPU and memory while raising questions about performance. Meet NVIDIA RTX PRO 6000 Blackwell — the most powerful desktop GPU ever built, ready to conquer advanced models and demanding creative workflows.GE Healthcare. NVIDIA Blackwell GPUs have been 2.5X faster than RTX 6000 Ada for AI training. The RTX Pro 6000 Blackwell for workstations will ship with 96GB of GDDR7 memory and needs 600 watts of power, slightly more than the 575 watts of the RTX 5090. The GPU even looks similar to the RTX 5090, and has double-flow-through cooling, 24,064 CUDA cores , a 512-bit memory... The Nvidia RTX Pro 6000 professional graphics card is based on the same GB202 graphics chip as the gaming RTX 5090. But it has more computing units — 24,064 shader c

## Built-in-tool ***Shell-Tool***

In [ ]:
from langchain_community.tools import ShellTool

st = ShellTool()

# print(st._run('pip show langchain'))
print(st.invoke('pip show langchain'))

"""Both the invoke and _run does the same thing"""

d:\Programing\LangChain\my_env\Lib\site-packages\langchain_community\tools\shell\tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


Executing command:
 pip show langchain
Name: langchain
Version: 0.3.27
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: D:\Programing\LangChain\my_env\Lib\site-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: langchain-community



There are others tools available as well. Visit Langchain official site for better understanding

# Custom Tools in LangChain

## Creating my own tool:

#### **Method-1** using `decorator` method

In [2]:
# importing library:
from langchain_core.tools import tool

In [ ]:
# Creating a simple tool for multiply two int:

@tool   #-> this decorator make the simple python function operable as tool in langchain, this also make the func a runnable obj
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return (a*b)

In [34]:
# try:
multiply.invoke({'a':10, 'b':15})

150

In [10]:
print(multiply.name)
print(multiply.args)
print(multiply.description)

multiply
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}
Multiply two numbers.


In [11]:
print(multiply.args_schema.model_json_schema())

{'description': 'Multiply two numbers.', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


#### **Method-2** using `pydantic+structured_tool` method

In [16]:
# Importing libraries:
from pydantic import BaseModel, Field
from langchain_core.tools import StructuredTool

In [27]:
# creating pydantic class:
class NumberCheck(BaseModel):
    a: int = Field(...,gt=0, description="Enter the number")

In [28]:
# define the function:
def check_even_odd(a):
    if a % 2 == 0:
        return "Even"
    else:
        return "Odd"

In [29]:
odd_even_tool = StructuredTool.from_function(
    func=check_even_odd,
    name="Odd-Even-Checker",
    description="Check whether a number is odd or even",
    args_schema=NumberCheck
)

In [33]:
odd_even_tool.invoke({'a':3354351351436})

'Even'

In [36]:
print(odd_even_tool.name)
print(odd_even_tool.description)
print(odd_even_tool.args)

Odd-Even-Checker
Check whether a number is odd or even
{'a': {'description': 'Enter the number', 'exclusiveMinimum': 0, 'title': 'A', 'type': 'integer'}}


#### **Method-3** using `BaseTool` class method

In [60]:
#import libraries:
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type

In [61]:
# creating pydantic class:
class NumberCheck(BaseModel):
    a: int = Field(...,gt=0, description="Enter the number")

In [71]:
# define the function using BaseTool:

class OddEvenTool(BaseTool):
    name: str = "Odd Even Checker"
    description: str = "Determines whether a number is Odd or Even"
    args_schema: Type[BaseModel] = NumberCheck
    
    def _run(self, a : int) -> str:
        if a % 2 == 0:
            return "Even"
        else:
            return "Odd" 
    


In [72]:
odd_even = OddEvenTool()

In [75]:
odd_even.invoke({'a': 111})

'Odd'